In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

DEBUG:root:test


In [59]:
import pandas as pd
store = pd.HDFStore('store.h5')

In [5]:
# Iex finance links:
# 0. https://addisonlynch.github.io/iexfinance/stable/stocks.html#stocks-historical
# 1. HistoricalReader class: https://github.com/addisonlynch/iexfinance/blob/d6d23537029895cc0ea62b25649aeeda67103b9e/iexfinance/stocks/historical.py
# 2. Stock class: https://github.com/addisonlynch/iexfinance/blob/d6d23537029895cc0ea62b25649aeeda67103b9e/iexfinance/stocks/base.py#L8
# 3. _IEXBase: https://github.com/addisonlynch/iexfinance/blob/b23c0646f614c84dc32750aee3bbb3cf10afbe35/iexfinance/base.py#L18

# https://intercom.help/iexcloud/en/articles/4063720-historical-stock-prices-on-iex-cloud
from iexfinance.stocks import get_historical_data

In [6]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
NUM_YEARS_HISTORY = 1
SYMBOL = 'AMZN'
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"

In [110]:
def sanitize_date(date):
    return datetime(date.year, date.month, date.day)

def get_historical_data_cached(store, symbol, start_date, end_date, **kwargs):
    start_date = sanitize_date(start_date)
    end_date = sanitize_date(end_date)
    # Assumptions:
    # 1. No missing data in existing dataframes in the store; data is complete between (date_min, date_max).
    # 2. The requested data is of the same format (URL, params, retrieved columns, etc...).
    # Reference for metadata: https://moonbooks.org/Articles/How-to-add-metadata-to-a-data-frame-with-pandas-in-python-/#store-in-a-hdf5-file
    if symbol not in store:
        print("here0")        

        df = get_historical_data(symbol, start_date, end_date, **kwargs)
        metadata = {'min_date': start_date,'max_date': end_date}
        
        store[symbol] = df        
        store.get_storer(symbol).attrs.metadata = metadata
    else:
        df = store[symbol]
        metadata = store.get_storer(symbol).attrs.metadata
        print(metadata)

        if start_date < metadata['min_date']:
            print("here1")
            df = df.append(get_historical_data(symbol, start_date, metadata['min_date'], **kwargs))
            metadata['min_date'] = start_date
        
        if end_date > metadata['max_date']:
            print("here2", end_date, metadata['max_date'])            
            df = df.append(get_historical_data(symbol, metadata['max_date'], end_date, **kwargs))
            metadata['max_date'] = end_date            

        df = df[~df.index.duplicated(keep='first')]        
        store[symbol] = df
        store.get_storer(symbol).attrs.metadata = metadata        
    return df

store = pd.HDFStore('store.h5')
end_date = datetime.now() #- relativedelta(weeks=10)
start_date = end_date - relativedelta(years=14) + relativedelta(days=10)
SYMBOL = 'AAPL'
get_historical_data_cached(store, SYMBOL, start_date, end_date, close_only=True, output_format='pandas', token=IEX_TOKEN)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443


here0


DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=max&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG - REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=max&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG:iexfinance.base:REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=max&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG - RESPONSE: 200
DEBUG:iexfinance.base:RESPONSE: 200
INFO - MESSAGES USED: 2518
INFO:iexfinance.base:MESSAGES USED: 2518
/Users/olshansky/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:43: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->bl

,close,volume
2015-11-16,28.83,153963583
2015-11-17,29.37,114976074
2015-11-18,30.65,187520691
2015-11-19,30,173606370
2015-11-20,30.58,138770411
...,...,...
2020-11-09,120.35,158666003
2020-11-10,117.69,144802655
2020-11-11,124.82,116021697
2020-11-12,124,105087444
